In [105]:
import pandas as pd
import numpy as np
import dash
import plotly.graph_objects as go
import plotly.express as px
import statistics
import plotly.io as pio
import nbformat
from dash import Dash, dcc, html, Input, Output
from plotly.subplots import make_subplots
import plotly.graph_objects as go


df_cr = pd.read_csv('../Data/Cash request.csv')
df_fe = pd.read_csv('../Data/Fees.csv')


1. **Frequency of Service Usage:** Understand how often users from each cohort utilize IronHack Payments' cash advance services over time.


In [106]:
# Let's calculate the number of unique ids of both deleted and actives
uni_ids_user = df_cr['user_id'].unique()
uni_ids_del = df_cr['deleted_account_id'].unique()
n_cr_per_u = len(df_cr['id'])

len_uni_ids_user = len(uni_ids_user)
len_uni_ids_del = len(uni_ids_del)
print(f"We have {len_uni_ids_del} deleted accounts and {len_uni_ids_user} id users.")
real_users = len_uni_ids_del + len_uni_ids_user
print(f"There are {real_users} users and {n_cr_per_u} cash requests")

# real_users is the colomn with the combined number of users
df_cr['real_users'] = df_cr['user_id'] + df_cr['deleted_account_id']


We have 1142 deleted accounts and 10799 id users.
There are 11941 users and 23970 cash requests


In [107]:

# Let's see the data in a visual way so we can decide if they are already clean or we need to clean out the outliers.

df_cr['real_users'] = df_cr['user_id'].combine_first(df_cr['deleted_account_id'])

# Let's create the new subdataset. The index will be users.

cr_per_user = df_cr.groupby('real_users')['id'].count().reset_index(name='n_cr_per_u')

# Let's see an histogram 
fig1 = px.histogram(cr_per_user, x='n_cr_per_u', nbins=5, title='Figure 1.1 Cash Requests by Users')
fig1.update_layout(xaxis_title='Number of Cash Request (per user)', yaxis_title='Number of Users')
fig1.show()

# And what about a boxplot to help understand this dataset?
fig2 = px.box(cr_per_user, y='n_cr_per_u', title='Figure 1.2 Boxplot of Cash Requests by User')
fig2.update_layout(yaxis_title='Number of Cash Requests')
fig2.show()

# And what about the average? 2 decimals
mean_cr_per_user = cr_per_user['n_cr_per_u'].mean()
print(f"The average requests by user are: {mean_cr_per_user:.2f}")



The average requests by user are: 2.03


In [108]:
# Can we agree that the best solution is a Histogram, right? There seems to be a lot of outliers, so we are gonna use only 5 bins.
# Do I need to know the outliers? Let's calculate
Q1 = cr_per_user['n_cr_per_u'].quantile(0.25)
Q3 = cr_per_user['n_cr_per_u'].quantile(0.75)
IQR = Q3 - Q1

# Define limits
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter outliers
df_cr_clean = cr_per_user[(cr_per_user['n_cr_per_u'] >= lower_bound) & (cr_per_user['n_cr_per_u'] <= upper_bound)]

df_cr_clean['real_users'] = df_cr['user_id'].combine_first(df_cr['deleted_account_id'])
df_cr_clean['real_users'].isnull().values.any() # it checks if there's missing values in data

# Let's see again the histogram 
fig13 = px.histogram(df_cr_clean, x='n_cr_per_u', nbins=5, title='Figure 1.3. Cash Requests by Users')
fig13.update_layout(xaxis_title='Number of Cash Requests', yaxis_title='Number of Users')
fig13.show()

# And the boxplot
fig14 = px.box(df_cr_clean, y='n_cr_per_u', title='Figure 1.4. Boxplot of Cash Requests by User')
fig14.update_layout(yaxis_title='Number of Cash Requests')
fig14.show()

# And what about the mean now? 2 decimals
mean_cr_per_user = df_cr_clean['n_cr_per_u'].mean()
print(f"The average requests by user are: {mean_cr_per_user:.2f}")

/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/1949531723.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



The average requests by user are: 1.39


In [ ]:
# So, now that we have everyting, let's calculate the frequency of cash requests over time, or the number of cash requests by month
# our number of real users is cr_per_user_clean, our axys y, and the axys x will be every month since the creation
# The best output is a line

# set as a datatime and extract the month and year for grouping
df_cr['created_at'] = pd.to_datetime(df_cr['created_at'])
df_cr['month_year'] = df_cr['created_at'].dt.to_period('M')

# I need the total by month
df_cr_clean = df_cr.groupby('month_year').size().reset_index(name='n_cr_per_month')
df_cr_clean['month_year'] = pd.to_datetime(df_cr_clean['month_year'].astype(str))  # Ensure datetime type
df_cr_clean['formatted_months'] = df_cr_clean['month_year'].dt.strftime('%B %Y')
df_cr_clean = df_cr_clean.sort_values(by='month_year')

# Create the plot with a distinct color palette
fig_cohort_frequency = px.line(
    df_cr_clean,
    x='formatted_months',
    y='n_cr_per_month',
  
    title='Frequency of Cash Advance Service Usage by Cohort Over Time',
    labels={
        'formatted_months': 'Months Since Cohort Creation',
        'n_cr_per_month': 'Number of Cash Requests',
    
    },
    hover_data={'formatted_months': True, 'n_cr_per_month': True},
    color_discrete_sequence=px.colors.qualitative.Dark24 # Use a qualitative palette for distinct colors
)

# Update layout for better readability
fig_cohort_frequency.update_layout(
    xaxis_title="Months Since Cohort Creation",
    yaxis_title="Number of Cash Requests"
)

fig_cohort_frequency.show()

# Ensure datetime and period columns
df_cr['created_at'] = pd.to_datetime(df_cr['created_at'])
df_cr['month_year'] = df_cr['created_at'].dt.to_period('M')

# Filter for the period inside the table
start = pd.Period('2019-11', freq='M')
end = pd.Period('2020-12', freq='M')
df_cr_period = df_cr[(df_cr['month_year'] >= start) & (df_cr['month_year'] <= end)]

# Assign quarter Q to each row
df_cr_period['trimester'] = df_cr_period['created_at'].dt.to_period('Q')

# Group by trimester and month, count requests
monthly_counts = df_cr_period.groupby(['trimester', 'month_year']).size().reset_index(name='n_cr_per_month')

# Calculate average requests per month for each trimester
avg_requests_per_trimester = monthly_counts.groupby('trimester')['n_cr_per_month'].mean().round(2).reset_index(name='avg_requests_per_month')

avg_requests_per_trimester['pct_increment'] = avg_requests_per_trimester['avg_requests_per_month'].pct_change().multiply(100).round(2)

print(avg_requests_per_trimester)


/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/3646741242.py:7: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



  trimester  avg_requests_per_month  pct_increment
0    2019Q4                  145.00            NaN
1    2020Q1                  217.00          49.66
2    2020Q2                 1308.33         502.92
3    2020Q3                 3746.33         186.34
4    2020Q4                 3932.50           4.97
The max number of cash request is 0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
Name: max_by_month, dtype: int64


/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/3646741242.py:41: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/3646741242.py:49: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [110]:
# Let's calculate now the first request made by every user group by month. Like that, we can know what is the attraction of the product.

# We redo everything in a copy of the original df
df_cr_fcr = df_cr
df_cr_fcr['real_users'] = df_cr['user_id'].combine_first(df_cr['deleted_account_id'])
df_cr_fcr['created_at'] = pd.to_datetime(df_cr['created_at'], errors='coerce')
df_cr_fcr['month_year'] = df_cr['created_at'].dt.to_period('M')
df_cr_fcr['formatted_months'] = df_cr['created_at'].dt.strftime('%B %Y')

# Sort by users and month of creation to ensure the first request is at the top for each user. Formatted months is a string, so let's use a datetime column
df_cr_sorted = df_cr_fcr.sort_values(by=['real_users', 'created_at'])

# Drop duplicates based on 'real_users' to keep only the first request for each user
df_cr_first_cr = df_cr_sorted.drop_duplicates(subset='real_users', keep='first')

# Finally, we group by month and then we get the amount
total_first_cr_amount_monthly = df_cr_first_cr.groupby('month_year')['real_users'].count().reset_index(name='total_first_cr_per_user')
total_first_cr_amount_monthly['formatted_months'] = total_first_cr_amount_monthly['month_year'].dt.strftime('%B %Y')

# Create the plot with a distinct color palette
fig_cohort_first_cr = px.line(
    total_first_cr_amount_monthly,
    x='formatted_months',
    y='total_first_cr_per_user',
  
    title='Number of First Cash Requests by Cohort Over Time',
    labels={
        'formatted_months': 'Months Since Cohort Creation',
        'total_first_cr_per_user': 'Number of First Cash Requests',
    },
    hover_data={'formatted_months': True, 'total_first_cr_per_user': True},
    color_discrete_sequence=px.colors.qualitative.Dark24 # Use a qualitative palette for distinct colors
)

# Update layout for better readability
fig_cohort_first_cr.update_layout(
    xaxis_title="Months Since Cohort Creation",
    yaxis_title="Number of First Cash Requests"
)

fig_cohort_first_cr.show()

# Let's repeat what we have done adapting the code
# Assign quarter to each row
df_cr_first_cr['trimester'] = df_cr_first_cr['created_at'].dt.to_period('Q')

# Group by quarter and count new users
quarterly_new_users = df_cr_first_cr.groupby('trimester')['real_users'].count().reset_index(name='n_new_users_per_quarter')

# Calculate number of months in each quarter
quarterly_new_users['months_in_quarter'] = quarterly_new_users['trimester'].apply(lambda x: len(pd.period_range(x.start_time, x.end_time, freq='M')))

# Calculate average new users per month in each quarter
quarterly_new_users['avg_new_users_per_month'] = (quarterly_new_users['n_new_users_per_quarter'] / quarterly_new_users['months_in_quarter']).round(2)

# Calculate percentage change quarter over quarter
quarterly_new_users['pct_increment'] = quarterly_new_users['avg_new_users_per_month'].pct_change().multiply(100).round(2)

print(quarterly_new_users)


/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/2463328924.py:7: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



  trimester  n_new_users_per_quarter  months_in_quarter  \
0    2019Q4                      255                  3   
1    2020Q1                      282                  3   
2    2020Q2                     2218                  3   
3    2020Q3                     4160                  3   
4    2020Q4                     4878                  3   

   avg_new_users_per_month  pct_increment  
0                    85.00            NaN  
1                    94.00          10.59  
2                   739.33         686.52  
3                  1386.67          87.56  
4                  1626.00          17.26  


/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/2463328924.py:45: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/2463328924.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2. **Incident Rate:** Determine the incident rate, specifically focusing on payment incidents, for each cohort. Identify if there are variations in incident rates among different cohorts.



In [111]:
# Let's check if all the users that had an incident also exist in the cash request column:
all_exist = df_fe['cash_request_id'].isin(df_cr['id']).all()
print(f"All cash_request_id in df_fe exist in df_cr['id']: {all_exist}")

missing_ids = df_fe[~df_fe['cash_request_id'].isin(df_cr['id'])]['cash_request_id'].unique()
print(f"Missing cash_request_id in {df_cr['id']}:", missing_ids)
missing_count = df_fe['cash_request_id'].isnull().value_counts()
print(f"There is a total of {missing_count} missing values, which is {missing_ids}")

# It's good to know, but the sample is so small that it won't affect the analysis.

All cash_request_id in df_fe exist in df_cr['id']: False
Missing cash_request_id in 0            5
1           70
2            7
3           10
4         1594
         ...  
23965    20616
23966    25243
23967    22357
23968    20256
23969    19886
Name: id, Length: 23970, dtype: int64: [nan]
There is a total of cash_request_id
False    21057
True         4
Name: count, dtype: int64 missing values, which is [nan]


In [112]:
# Let's create a dataframe with the columns: id, type, status, category
df_incidents = df_fe[df_fe['type'] == 'incident'][['id', 'type', 'status', 'category']]
display(df_incidents)

# The length of the cash resquest id is n_cr_per_u
# Let's use the data that is interesting for us:
n_non_incidents = df_fe['id'].count() - df_fe[df_fe['type'] == 'incident']['id'].count()
n_incidents = df_fe[df_fe['type'] == 'incident']['id'].count()
n_fees = df_fe['id'].count()
print(f"The number of incidents is {n_incidents}, the number of non-incidents is {n_non_incidents} and the total number of fees is {n_fees}.")

# Let's calculate the rate of incidents for all the fees:

incident_rate = n_incidents / n_fees
print(f"The incident rate regarding only fees is: {incident_rate:.2%}.")

# Now, let's calculate the rate of incidents for all the cash request:
incident_rate = n_incidents / n_cr_per_u
print(f"Only a {incident_rate:.2%} of the total cash requests (a total of {n_cr_per_u}) has got any incidents.")


,id,type,status,category
1,6961,incident,accepted,rejected_direct_debit
7,14187,incident,accepted,rejected_direct_debit
15,14420,incident,accepted,rejected_direct_debit
27,14433,incident,rejected,rejected_direct_debit
40,14188,incident,accepted,rejected_direct_debit
...,...,...,...,...
21044,3491,incident,rejected,rejected_direct_debit
21045,7166,incident,rejected,month_delay_on_payment
21046,12566,incident,rejected,month_delay_on_payment
21047,13537,incident,rejected,rejected_direct_debit


The number of incidents is 2196, the number of non-incidents is 18865 and the total number of fees is 21061.
The incident rate regarding only fees is: 10.43%.
Only a 9.16% of the total cash requests (a total of 23970) has got any incidents.


In [113]:
# Let's make it fun and to create a dataframe out of it:

# We choose the relevant columns and we change the name to distinguish them:
df_cr_sub = df_cr[['real_users', 'id']].rename(columns={'id': 'id_cr'})
df_fe_sub = df_fe[['id', 'cash_request_id', 'type']].rename(columns={'id': 'id_fe'})

# We combine using the ids.
df_combined = pd.merge(df_cr_sub, df_fe_sub, left_on='id_cr', right_on='id_fe', how='left')

# Our index will be the users, as before.
df_combined.set_index('real_users', inplace=True)

print(df_combined.head())

            id_cr   id_fe  cash_request_id      type
real_users                                          
804.0           5     5.0           2568.0  postpone
231.0          70    70.0           1796.0  postpone
191.0           7     7.0           1518.0  postpone
761.0          10    10.0           2278.0  postpone
7686.0       1594  1594.0           4065.0  incident


In [114]:
# Total requests per user
total_requests = df_combined.groupby(df_combined.index)['id_cr'].count()

# Incidents per user
incidents_per_user = df_combined[df_combined['type'] == 'incident'].groupby(df_combined[df_combined['type'] == 'incident'].index)['id_cr'].count()

# Rate per user (fillna(0) for users with no incidents)
incident_rate_per_user = (incidents_per_user / total_requests).fillna(0)

print(incident_rate_per_user.head())

real_users
34.0    0.000
35.0    0.000
47.0    0.125
52.0    0.000
53.0    0.000
Name: id_cr, dtype: float64


In [154]:
# Let's make it fancy. 
fig = px.histogram(
    incident_rate_per_user,
    x=incident_rate_per_user.values,
    nbins=2,
    title='Figure 2 Distribution of the rate of incidents by users'
)
fig.update_layout(
    xaxis_title='Rate of incidents by user',
    yaxis_title='Number of users'
)
fig.show()

# Maybe we don't need a figure for this rate, but we are happy to see that the most of the users
# didn't have any incident.

# Let's verify the variations in incident rates among different cohorts.

Analyzing the data, we see that incidents are not related to a specific moment in time. 

Let's see which are the user that had the most incidents and the user with the least incidents:
The user with most incidents was the id 1909 with 8 incidents, the least is not interesting since the most of them only had 1 incident.



In [116]:
# Merge to relate both columns
merged = pd.merge(
    df_cr[['id', 'real_users']],
    df_fe[['cash_request_id', 'type']],
    left_on='id',
    right_on='cash_request_id',
    how='inner'
)

# Filter by incidents
incidents = merged[merged['type'] == 'incident']

# Count incidents by user
incidents_per_user = incidents.groupby('real_users').size()

# User with more incidents
user_most = incidents_per_user.idxmax()
max_incidents = incidents_per_user.max()

# User with less incidents (at least 1)
user_least = incidents_per_user.idxmin()
min_incidents = incidents_per_user.min()

print(f"User with most incidents: {user_most} ({max_incidents} incidents)")
print(f"User with least incidents: {user_least} ({min_incidents} incident)")




User with most incidents: 1909.0 (8 incidents)
User with least incidents: 72.0 (1 incident)


3. **Revenue Generated by the Cohort:** Calculate the total revenue generated by each cohort over months to assess the financial impact of user behavior.

I have the total number of users. The amount of every id cash request (that I can calculate), I can also sum all the fees. Should I substract anything? 
Or filter? All of the types are concerned, only fees in status accepted, 
Others? We can relate fees and cash requests. How much money do we get out of all the cash request? Ex. 10$ in fees by CR
Calculate:
3.1 The total amount of fees that are received (status accepted)
The total amount of cash that's been approved, money-sent, active MINUS money_back, 


In [ ]:
# In this cell, I am preparing the table with the ingredients
# First, the total amount and the total amount of fees accepted, ie they were successfully charged
total_fees = df_fe[df_fe['status'] == 'accepted']['total_amount'].sum()
print(f'The total amount received from accepted fees is of {total_fees} (for status accepted).')
total_fees_two = df_fe['total_amount'].sum()
print(f'The total amount received from all fees (all statuses) is of {total_fees_two}.')

# 1. Get cohort (first cash request month) for each user
df_cr['created_at'] = pd.to_datetime(df_cr['created_at'], errors='coerce', format='mixed')
df_cr['first_request_month'] = df_cr['created_at'].dt.to_period('M').astype(str)
user_cohort = df_cr.groupby('real_users')['first_request_month'].min().reset_index()

# 2. Merge with fees and users
cr_cols = ['real_users', 'id', 'amount', 'status']
fe_cols = ['cash_request_id', 'total_amount', 'type', 'paid_at', 'status']
df_merged = pd.merge(
    df_cr[cr_cols],
    df_fe[fe_cols],
    left_on='id',
    right_on='cash_request_id',
    how='left'
)
df_merged = pd.merge(df_merged, user_cohort, on='real_users', how='left')

# 3. Filter only accepted fees and valid paid_at
df_merged = df_merged[(df_merged['status_y'] == 'accepted') & df_merged['paid_at'].notnull()]

# Extract month of fee
df_merged['paid_at'] = pd.to_datetime(df_merged['paid_at'], errors='coerce', format='mixed')
df_merged['month_of_fee'] = df_merged['paid_at'].dt.to_period('M').astype(str)
df_merged['month_of_fee'] = pd.to_datetime(df_merged['paid_at']).dt.to_period('M').astype(str)

# 5. Group by cohort and month, sum revenue
revenue_by_cohort = df_merged.groupby(['first_request_month', 'month_of_fee'])['total_amount'].sum().reset_index()

# 6 Revenue of fees by month
revenue_by_month = df_merged.groupby(['month_of_fee'])['total_amount'].sum().reset_index()

## 6.1 Amount of money owed from cash requests
## Even if the values for the statuses approved and money_sent are missing, I want to add them.
statuses = ['approved', 'money_sent', 'active']
owed_by_month = df_merged[df_merged['status_x'].isin(statuses)].groupby(['month_of_fee', 'status_x'])['amount'].sum().reset_index()

## Sort by 'month_of_fee' as datetime for correct chronological order
revenue_by_month = revenue_by_month.sort_values(by='month_of_fee', key=lambda x: pd.to_datetime(x, format='%Y-%m'))

# 7 Revenue of fees by type
revenue_by_type = df_merged.groupby(['type'])['total_amount'].sum().reset_index()
print(revenue_by_cohort)
print(revenue_by_month)
print(revenue_by_type)



The total amount received from accepted fees is of 74210.0 (for status accepted).
The total amount received from all fees (all statuses) is of 105310.0.


/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/3647271668.py:8: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/3647271668.py:28: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/3647271668.py:29: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



   first_request_month month_of_fee  total_amount
0              2019-11      2020-07           5.0
1              2019-11      2020-09          10.0
2              2019-11      2020-10          10.0
3              2019-11      2020-11           5.0
4              2019-12      2020-06          10.0
..                 ...          ...           ...
88             2020-10      2021-02         285.0
89             2020-11      2020-11         170.0
90             2020-11      2020-12         125.0
91             2020-11      2021-01          20.0
92             2020-11      2021-02           5.0

[93 rows x 3 columns]
  month_of_fee  total_amount
0      2020-06         710.0
1      2020-07        2895.0
2      2020-08        3430.0
3      2020-09        9165.0
4      2020-10       20630.0
5      2020-11       27575.0
6      2020-12        5975.0
7      2021-01        1530.0
8      2021-02         710.0
              type  total_amount
0         incident        7255.0
1  instant_payment   

In [118]:
# Let's do a bar chart with a plot line so we can visualize this better, for month and revenues:

# Get all unique months, sorted
## Create formatted labels for the y-axis so it has the name of the month in it
formatted_months = pd.to_datetime(
    revenue_by_month['month_of_fee'], format='%Y-%m').sort_values().dt.strftime('%B %Y').unique()

# Bar + Line
fig = go.Figure()

# Bar chart for revenue
fig.add_trace(go.Bar(
    x=formatted_months,
    y=revenue_by_month['total_amount'],
    name='Revenue (Bar)',
    marker=dict(color='blue', opacity=0.5)
))

# Bar chart for money requested (owed)
fig.add_trace(go.Bar(
    x=formatted_months,
    y=owed_by_month['amount'],
    name='Money requested (Bar)',
    marker=dict(color='red', opacity=0.5)
))

# Line for revenues out of fees
fig.add_trace(go.Scatter(
    x=formatted_months,
    y=revenue_by_month['total_amount'],
    name='Revenue (Line)',
    mode='lines+markers',
    line=dict(color="#081661")
))

# Line for cash requested
fig.add_trace(go.Scatter(
    x=formatted_months,
    y=owed_by_month['amount'],
    name='Requested (Line)',
    mode='lines+markers',
    line=dict(color="#851111")
))

fig.update_layout(
    barmode='stack',
    title="Figure 3.1. Revenues by month (Stacked)",
    xaxis_title="Month of every money transfer",
    yaxis_title="Amount of revenues",
    xaxis=dict(
        tickmode='array',
        tickvals=formatted_months,
        ticktext=formatted_months,
        title='Month of every fee'
    )
)

fig.show()

In [167]:
fig = go.Figure()

# Revenue (Bar) on primary y-axis
fig.add_trace(go.Bar(
    x=formatted_months,
    y=revenue_by_month['total_amount'],
    name='Revenue (Bar)',
    marker=dict(color='blue', opacity=0.5),
    yaxis='y'
))

# Money requested (Bar) on secondary y-axis
fig.add_trace(go.Bar(
    x=formatted_months,
    y=owed_by_month['amount'],
    name='Money requested (Bar)',
    marker=dict(color='red', opacity=0.5),
    yaxis='y2'
))


# Line for revenues out of fees
fig.add_trace(go.Scatter(
    x=formatted_months,
    y=revenue_by_month['total_amount'],
    name='Revenue (Line)',
    mode='lines+markers',
    line=dict(color="#0F2BC6")
))

# Line for cash requested
fig.add_trace(go.Scatter(
    x=formatted_months,
    y=owed_by_month['amount'],
    name='Requested (Line)',
    mode='lines+markers',
    line=dict(color="#CC2B2B")
))

fig.update_layout(
    title="Figure 3.2. Revenues by month (2000)",
    xaxis_title="Month of every money transfer",
    yaxis=dict(
        title=dict(text="Revenue (k)", font=dict(color="black")),
        tickfont=dict(color="black"),
        side='left',
        range=[0, 30000],
        dtick=5000,
        tickmode='linear'
    ),
    yaxis2=dict(
        title=dict(text="Money requested", font=dict(color="black")),
        tickfont=dict(color="black"),
        overlaying='y',
        side='right',
        range=[0, 3000],
        dtick=500,
        tickmode='linear'
    ),
    xaxis=dict(
        tickmode='array',
        tickvals=formatted_months,
        ticktext=formatted_months
    ),
    barmode='group'
)

fig.update_layout(
    updatemenus = [go.layout.Updatemenu(
        active = 0,
        buttons = [
            go.layout.updatemenu.Button(
                label = "Histogram by Revenues",
                method = "update",
                args = [{"visible" : [True, False, False, False]}]),
            go.layout.updatemenu.Button(
                label = "Histogram by Money Requested",
                method = "update",
                args = [{"visible" : [False, True, False, False]}]),
            go.layout.updatemenu.Button(
                label = "Line of Revenues and Money Requested",
                method = "update",
                args = [{"visible" : [False, False, True, True]}]),
            go.layout.updatemenu.Button(
                label = "All",
                method = "update",
                args = [{"visible" : [True, True, True, True]}])
        ]
    )]
)

fig.show()



In [ ]:

# Let's see the layout in two subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Revenue by Month", "Money Requested by Month"))

fig.add_trace(go.Bar(
    x=formatted_months,
    y=revenue_by_month['total_amount'],
    name='Revenue (Bar)',
    marker=dict(color='blue', opacity=0.5)
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=formatted_months,
    y=revenue_by_month['total_amount'],
    name='Revenue (Line)',
    mode='lines+markers',
    line=dict(color="#0F2BC6")
), row=1, col=1)

fig.add_trace(go.Bar(
    x=formatted_months,
    y=owed_by_month['amount'],
    name='Money requested (Bar)',
    marker=dict(color='red', opacity=0.5)
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=formatted_months,
    y=owed_by_month['amount'],
    name='Requested (Line)',
    mode='lines+markers',
    line=dict(color="#CC2B2B")
), row=1, col=2)

fig.update_layout(
    title="Figure 3.3. Revenues and Money Requested by Month (Separated)",
    xaxis_title="Month of every reimbursement",
    yaxis_title="Revenue (k)",
    xaxis2_title="Month of every money transfer",
    yaxis2_title="Money requested",
    showlegend=False
)

fig.show()

# New Relevant Metrics:

4.1. User Retention Rate per Cohort

Definition:
The retention rate measures the percentage of users from each cohort who make at least one additional cash request in a later month after their first request. This helps you understand user engagement and loyalty over time.

How to calculate it:
Assign cohort: Use the month of the user's first cash request as their cohort.
For each cohort:
Count the number of users who made at least one cash request in a later month (not just their first month).
Divide by the total number of users in that cohort.

In [123]:
# Assign cohort (first request month)
df_cr['created_at'] = pd.to_datetime(df_cr['created_at'])
df_cr['cohort'] = df_cr.groupby('real_users')['created_at'].transform('min').dt.to_period('M').astype(str)
df_cr['request_month'] = df_cr['created_at'].dt.to_period('M').astype(str)

# Users who made requests in later months
df_cr['is_retained'] = df_cr['cohort'] != df_cr['request_month']

# For each user, check if they made a request in a later month
user_retention = df_cr.groupby(['real_users', 'cohort'])['is_retained'].any().reset_index()

# Calculate total users and retained users per cohort
cohort_stats = user_retention.groupby('cohort').agg(
    total_users=('real_users', 'nunique'),
    retained_users=('is_retained', 'sum')
).reset_index()

# Optionally, add the next month for each cohort
cohort_stats['month_of_RR'] = pd.to_datetime(cohort_stats['cohort'], format='%Y-%m').dt.to_period('M') + 1
cohort_stats['month_of_RR'] = cohort_stats['month_of_RR'].astype(str)

# Calculate retention rate (percentage)
cohort_stats['retention_rate'] = cohort_stats['retained_users'] / cohort_stats['total_users'] * 100

print(cohort_stats)

     cohort  total_users  retained_users month_of_RR  retention_rate
0   2019-11            1               1     2019-12      100.000000
1   2019-12          254             174     2020-01       68.503937
2   2020-01          106              65     2020-02       61.320755
3   2020-02           90              66     2020-03       73.333333
4   2020-03           86              66     2020-04       76.744186
5   2020-04          246             166     2020-05       67.479675
6   2020-05          432             344     2020-06       79.629630
7   2020-06         1540            1136     2020-07       73.766234
8   2020-07         1482             971     2020-08       65.519568
9   2020-08          728             398     2020-09       54.670330
10  2020-09         1950             694     2020-10       35.589744
11  2020-10         4802              15     2020-11        0.312370
12  2020-11           76               0     2020-12        0.000000


/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/1962476463.py:3: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/1962476463.py:4: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [160]:

fig = go.Figure()

# Line for percent of rate
fig.add_trace(go.Scatter(
    x=cohort_stats['month_of_RR'].iloc[2:13],
    y=cohort_stats['retention_rate'],
    name='(%)',
    mode='lines+markers+text',
    text=[f"{v:.0f}%" for v in cohort_stats['retention_rate']],
    textposition='top center',
    line=dict(color="#C5840B")
))

fig.add_trace(go.Bar(
    x=cohort_stats['cohort'].iloc[2:13],
    y=cohort_stats['total_users'],
    name='Total Users',
    marker=dict(color='blue', opacity=0.6),
    yaxis='y2' 
))

fig.add_trace(go.Bar(
    x=cohort_stats['cohort'].iloc[2:13],
    y=cohort_stats['retained_users'],
    name='Retained users',
    marker=dict(color='purple', opacity=0.6),
    yaxis='y2' # Assigned to same y axis so it superpose it
))

fig.add_annotation(
    text="This chart shows the retention rate per cohort for the previous month.",
    xref="paper", yref="paper",
    x=-0.05, y=1.15, showarrow=False,
    font=dict(size=12)

)

fig.update_layout(
    title="Retention Rates (%) in the Later Month in 2020",
    xaxis_title="2020 Period",
    yaxis=dict(
        showticklabels=False,  # Hide tick labels
        showgrid=False,        # Hide grid lines
        ticks='',              # Hide tick marks
        zeroline=False         # Hide zero line
    ),
    yaxis2=dict(
        title=dict(text="Number of Users", font=dict(color="black")),
        tickfont=dict(color="black"),
        range=[0, 2000],
        dtick=500,
        tickmode="linear",
        overlaying='y',
        side='left'
    ),
    
    xaxis=dict(
        tickmode='array',
        tickvals=cohort_stats['cohort'].iloc[3:13],
        ticktext=cohort_stats['cohort'].iloc[3:13],
        title='2020 Period'
    ),
    barmode='overlay',

)

fig.show()


**4.2. Average Revenue Per User (ARPU) per Cohort**

**Definition:**  
ARPU measures the average amount of revenue generated per user in each cohort over a given period. This helps you understand the financial value of your user base and compare cohorts’ profitability.

**How to calculate it:**

1. Assign each user to a cohort (e.g., by their first cash request month).
2. For each cohort, sum the total revenue generated by users in that cohort.
3. Divide by the number of users in the cohort.

In [128]:
# Assign cohort (first request month)
df_cr['created_at'] = pd.to_datetime(df_cr['created_at'])
df_cr['cohort'] = df_cr.groupby('real_users')['created_at'].transform('min').dt.to_period('M').astype(str)

# Merge fees with users and cohort
cr_cols = ['real_users', 'id', 'cohort']
fe_cols = ['cash_request_id', 'total_amount', 'status']
df_merged = pd.merge(
    df_cr[cr_cols],
    df_fe[fe_cols],
    left_on='id',
    right_on='cash_request_id',
    how='left'
)
df_merged = df_merged[df_merged['status'] == 'accepted']

# Calculate total revenue and user count per cohort
cohort_revenue = df_merged.groupby('cohort')['total_amount'].sum()
cohort_users = df_cr.groupby('cohort')['real_users'].nunique()

# Calculate ARPU per cohort
arpu_per_cohort = (cohort_revenue / cohort_users).reset_index(name='ARPU')

print(arpu_per_cohort)

     cohort       ARPU
0   2019-11  30.000000
1   2019-12   4.448819
2   2020-01   4.339623
3   2020-02   7.055556
4   2020-03   7.500000
5   2020-04   6.443089
6   2020-05   9.097222
7   2020-06   9.327922
8   2020-07   7.513495
9   2020-08   6.929945
10  2020-09   5.846154
11  2020-10   4.900042
12  2020-11   4.210526


/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/1586395256.py:3: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [129]:
# Plot ARPU per cohort as a bar chart
fig = px.bar(arpu_per_cohort.iloc[2:], x="cohort", y="ARPU", title="ARPU per Cohort")
fig.update_layout(
	xaxis_title="Cohort in 2020",
	yaxis_title="Average Revenue Per User (ARPU) in %",
	xaxis=dict(
		tickmode='array',
		tickvals=arpu_per_cohort['cohort'],
		ticktext=arpu_per_cohort['cohort']
	)
)
fig.show()

4.3. Average time for closing an payment incident by type.
- Let's compare the money that has been recovered or pending, and it's been recovered or not using the last three columns.

In [130]:
# Clean the Nans in recovery_status and in the dates that we are interested
df_clean = df_cr.dropna(subset=['recovery_status', 'reco_creation', 'reco_last_update', 'amount'])

# We need to clean the data for making operations ouf of it
df_clean['reco_creation'] = pd.to_datetime(df_clean['reco_creation'])
df_clean['reco_last_update'] = pd.to_datetime(df_clean['reco_last_update'])
df_clean['days_to_close'] = (df_clean['reco_last_update'] - df_clean['reco_creation']).dt.days

# Find the tyme of closing in days using dt.days
df_clean['days_to_close'] = (df_clean['reco_last_update'] - df_clean['reco_creation']).dt.days

# Group by recovery_status and calculate average days to close and count
IncidentsbyStatuses = df_clean.groupby('recovery_status').agg(
    avg_days_to_close=('days_to_close', 'mean'),
    count=('days_to_close', 'size')
).reset_index()

fig = px.scatter(
    IncidentsbyStatuses,
    y="recovery_status",
    x="avg_days_to_close",
    
    symbol="count",
    color="count",
    labels={
        "avg_days_to_close": f"Average Days to Close",
        "recovery_status": "Recovery Status",
        "count": "Count"
    },
    title="Scatterplot of Incident Statuses"
)
fig.update_traces(marker_size=10)
fig.show()

/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/2265877759.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/2265877759.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/l9/5y25yg4122vbwsln35vlhw7c0000gn/T/ipykernel_25792/2265877759.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

# Result:
1. **Frequency of Service Usage:** Understand how often users from each cohort utilize IronHack Payments' cash advance services over time.

**There's an average cash request by user of 1.39.**

Cleaning the outliers was necessary, since the most of the cash requests were in the same quantile. 
We have 1142 deleted accounts and 10799 id users.
There are 11941 users and 23970 cash requests 

2. **Incident Rate:** Determine the incident rate, specifically focusing on payment incidents, for each cohort. Identify if there are variations in incident rates among different cohorts.

The incident rate regarding only fees is of 10.43%, but it will be more adequate to relate it directly with the cash requests, **with a rate incident of 9.16% over a total of 23970**.

The number of incidents is 2196, the number of non-incidents is 18865 and the total number of fees is 21061.
The incident rate regarding only fees is: 10.43%.
Only a 9.16% of the total cash requests (a total of 23970) has got any incidents.

The user with most incidents is 1909.0 with (8 incidents)

3. **Revenue Generated by the Cohort:** Calculate the total revenue generated by each cohort over months to assess the financial impact of user behavior.

To know the total revenue by month, you can refer to df revenue_by_month or to the bar chart (Figure 1). If you need to knoe the revenues by type of fee, refer to Figure 2. 
You can compare the total money requested and the money earned through fees in both figures 1 and 1.bis, with a different y-axis.

Finally: 
    The total amount received from accepted fees is of 74210.0 (for status accepted).
    The total amount received from all fees (all statuses) is of 105310.0.  

4. **New Relevant Metric:** Propose and calculate a new relevant metric that provides additional insights into user behavior or the performance of IronHack Payments' services.
- 4.1. Retention Rates in the last month of 2020

    The retention rate measures the percentage of users from each cohort who make at least one additional cash request in a later month after their first request. This helps you understand user engagement and loyalty over time.
- 4.2. Average Revenue per User (ARPU) per Cohort

    ARPU measures the average amount of revenue generated per user in each cohort over a given period. This helps you understand the financial value of your user base and compare cohorts’ profitability.
- 4.3. Average Time for closing an payment incident by type

    It measured the average time that is needed to close an incident of reimbursement in cash requests. Calculated by substracting the date of creation to the day of the last update, also known as the incident closure date.


